## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-28-01-00-00 +0000,wsj,The Hottest High Schools in Massachusetts Are ...,https://www.wsj.com/us-news/education/massachu...
1,2025-12-28-00-58-11 +0000,nyt,Ski Slopes Are Empty as a Labor Dispute Shuts ...,https://www.nytimes.com/2025/12/27/us/tellurid...
2,2025-12-28-00-50-15 +0000,nypost,"‘Unlucky’ illegal immigrant runs red light, cr...",https://nypost.com/2025/12/27/us-news/unlucky-...
3,2025-12-28-00-34-29 +0000,nypost,Bronx girl dies after being found unconscious ...,https://nypost.com/2025/12/27/us-news/bronx-gi...
4,2025-12-28-00-21-45 +0000,nypost,Ex-reality TV swinger jailed on child sex abus...,https://nypost.com/2025/12/27/us-news/former-r...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2366/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
86,trump,17
57,new,11
167,christmas,10
107,before,9
299,year,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
134,2025-12-27-03-30-11 +0000,nypost,Trump-Kennedy Center filing $1M lawsuit agains...,https://nypost.com/2025/12/26/us-news/trump-ke...,56
116,2025-12-27-09-04-51 +0000,nypost,"Russian drones, missiles pound Ukraine before ...",https://nypost.com/2025/12/27/world-news/russi...,55
124,2025-12-27-07-34-22 +0000,nyt,Zelensky Will Meet With Trump Over the Weekend...,https://www.nytimes.com/2025/12/26/world/europ...,51
119,2025-12-27-08-44-24 +0000,wapo,"Zelensky says he will meet Trump soon, citing ...",https://www.washingtonpost.com/world/2025/12/2...,50
139,2025-12-27-03-00-00 +0000,wsj,The U.S. is fast-tracking power projects for d...,https://www.wsj.com/politics/policy/ai-state-f...,50


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
134,56,2025-12-27-03-30-11 +0000,nypost,Trump-Kennedy Center filing $1M lawsuit agains...,https://nypost.com/2025/12/26/us-news/trump-ke...
99,45,2025-12-27-10-32-03 +0000,cbc,"Thailand, Cambodia sign new ceasefire deal to ...",https://www.cbc.ca/news/world/thailand-cambodi...
116,38,2025-12-27-09-04-51 +0000,nypost,"Russian drones, missiles pound Ukraine before ...",https://nypost.com/2025/12/27/world-news/russi...
139,33,2025-12-27-03-00-00 +0000,wsj,The U.S. is fast-tracking power projects for d...,https://www.wsj.com/politics/policy/ai-state-f...
42,31,2025-12-27-20-02-17 +0000,nypost,Mayor Adams turns Christmas time into Valentin...,https://nypost.com/2025/12/27/us-news/mayor-ad...
9,25,2025-12-28-00-00-00 +0000,wsj,President Trump overrode some on the religious...,https://www.wsj.com/politics/policy/how-trump-...
45,23,2025-12-27-19-46-16 +0000,cbc,"Jeffrey R. Holland, next in line to lead Churc...",https://www.cbc.ca/news/world/jeffrey-r-hollan...
35,22,2025-12-27-21-10-38 +0000,nypost,Ukrainian anti-corruption police raid Kyiv par...,https://nypost.com/2025/12/27/world-news/ukrai...
68,22,2025-12-27-15-00-00 +0000,nypost,Test your 2025 news knowledge with The Post’s ...,https://nypost.com/2025/12/27/lifestyle/test-y...
3,21,2025-12-28-00-34-29 +0000,nypost,Bronx girl dies after being found unconscious ...,https://nypost.com/2025/12/27/us-news/bronx-gi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
